In [28]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Creating the fully DNN

In [29]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes, *args, **kwargs) -> None:
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#model = NN(784, 10)
#x = torch.randn(64, 784)
#
#print(model(x).shape)

In [30]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10, *args, **kwargs) -> None:
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self,x):
        x=F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

#model = CNN()
#x = torch.randn(64,1,28,28)
#print(x.shape)

torch.Size([64, 1, 28, 28])


# Set device

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We run on a "+str(device))

We run on a cpu


# Hyperparameters

In [32]:
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

# Load data

In [33]:
train_dataset = datasets.MNIST(root="dataset/", train = True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root="dataset/", train = False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


# Initialize network

In [34]:
model = CNN().to(device)

# Loss and optimizer

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train network

In [36]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader) :
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x3136 and 784x10)

# Check the result

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking on training data")
    else :
        print("checking on test data")
    num_corrects = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_corrects+=(predictions==y).sum()
            num_samples+=predictions.size(0)

        print(f'{num_corrects} / {num_samples} with accuracy {float(num_corrects)/float(num_samples)*100:.2f}')
    model.train()    

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)      